In [1]:
!pip install transformers datasets ipywidgets peft bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.1 MB/s eta 0:00:00


In [2]:
import os
import math
import numpy as np
import torch
from tqdm.auto import tqdm 
from datetime import timedelta
import time
import gc
from peft import prepare_model_for_kbit_training

# Set memory optimization environment variable
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    TrainerCallback
)
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

# Free up memory before starting
torch.cuda.empty_cache()
gc.collect()

# 1. Load your text dataset from the Kaggle input path
with open('/kaggle/input/nbody-data/cleaned.md', 'r', encoding='utf-8') as f:
    corpus = f.read()

# 2. Load the tokenizer and determine the model's maximum context length
model_name = "Qwen/Qwen2.5-Math-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_context_length = tokenizer.model_max_length
print(f"Maximum context length: {max_context_length}")

# Use smaller chunks to further reduce memory pressure
chunk_size = 2048  # Further reduced from 4096

# 3. Tokenize the entire corpus and split it into reasonably sized chunks
def prepare_corpus_for_training(corpus, tokenizer, chunk_size):
    tokens = tokenizer(corpus, truncation=False, return_tensors="np")["input_ids"][0]
    
    total_chunks = math.ceil(len(tokens) / chunk_size)
    print(f"Total tokens: {len(tokens)}, Creating {total_chunks} chunks of size {chunk_size}")
    
    chunks = []
    for i in range(0, len(tokens), chunk_size):
        chunk = tokens[i:i + chunk_size].tolist()
        if len(chunk) < chunk_size:
            chunk = chunk + [tokenizer.pad_token_id] * (chunk_size - len(chunk))
        chunks.append({"input_ids": chunk})
    
    return Dataset.from_list(chunks)

chunked_dataset = prepare_corpus_for_training(corpus, tokenizer, chunk_size)

# 4. Create a unified progress tracking callback
class EnhancedProgressCallback(TrainerCallback):
    def __init__(self):
        self.training_start = time.time()
        self.epoch_start = None
        self.progress_bar = None
        self.current_epoch = 0
        
    def on_train_begin(self, args, state, control, **kwargs):
        print(f"\n{'='*70}")
        print(f"TRAINING STARTED")
        print(f"{'='*70}")
        
        # Calculate total steps for all epochs
        self.total_steps = state.max_steps
        self.steps_per_epoch = len(trainer.train_dataset) // (args.per_device_train_batch_size * 
                                                            args.gradient_accumulation_steps * 
                                                            torch.cuda.device_count())  # Account for multi-GPU
    
    def on_epoch_begin(self, args, state, control, **kwargs):
        self.epoch_start = time.time()
        self.current_epoch = state.epoch + 1
        
        print(f"\n{'='*70}")
        print(f"Beginning Epoch {self.current_epoch}/{args.num_train_epochs}")
        print(f"{'='*70}")
        
        # Create a progress bar for this epoch
        self.progress_bar = tqdm(
            total=self.steps_per_epoch, 
            desc=f"Epoch {self.current_epoch}/{args.num_train_epochs}",
            position=0
        )
        self.last_logged_step = 0
        
    def on_epoch_end(self, args, state, control, **kwargs):
        # Close progress bar
        if self.progress_bar:
            self.progress_bar.close()
        
        # Calculate epoch time
        epoch_time = time.time() - self.epoch_start
        total_time = time.time() - self.training_start
        
        print(f"\n{'='*70}")
        print(f"Completed Epoch {self.current_epoch}/{args.num_train_epochs}")
        print(f"Epoch time: {timedelta(seconds=int(epoch_time))}")
        print(f"Total training time: {timedelta(seconds=int(total_time))}")
        
        # Estimate remaining time
        epochs_remaining = args.num_train_epochs - self.current_epoch
        est_remaining = epoch_time * epochs_remaining
        print(f"Estimated time remaining: {timedelta(seconds=int(est_remaining))}")
        print(f"{'='*70}\n")
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and logs and self.progress_bar:
            # Calculate step progress
            current_step_in_epoch = state.global_step % self.steps_per_epoch
            if current_step_in_epoch == 0 and state.global_step > 0:
                current_step_in_epoch = self.steps_per_epoch
                
            # Update progress bar to current position
            self.progress_bar.n = current_step_in_epoch
            
            # Add metrics to progress bar
            postfix_dict = {}
            if "loss" in logs:
                postfix_dict["loss"] = f"{logs['loss']:.4f}"
            if "learning_rate" in logs:
                postfix_dict["lr"] = f"{logs['learning_rate']:.2e}"
            
            # Add GPU memory usage
            try:
                allocated = torch.cuda.memory_allocated() / (1024 ** 3)
                postfix_dict["GPU"] = f"{allocated:.1f}GB"
            except:
                pass
                
            self.progress_bar.set_postfix(**postfix_dict)
            self.progress_bar.update(0)  # Force refresh
            
            # Print step details with percentage completed
            current_overall = state.global_step
            progress_percent = current_overall / self.total_steps * 100
            if current_overall % 20 == 0:  # Print every 20 steps
                print(f"Step: {current_overall}/{self.total_steps} ({progress_percent:.1f}%) | "
                      f"Loss: {logs.get('loss', 0):.4f} | "
                      f"LR: {logs.get('learning_rate', 0):.2e}")

# Create data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 5. LoRA-optimized training arguments for continued pretraining
training_args = TrainingArguments(
    output_dir="./qwen_math_nbody_lora",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    # Standard learning rate for non-embedding layers
    learning_rate=1e-4,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,
    dataloader_drop_last=True,
    report_to=["tensorboard"],
    fp16=True,
    logging_first_step=True,
    gradient_checkpointing=True,
    logging_dir="./logs",
    warmup_steps=100,
    logging_strategy="steps",
    gradient_checkpointing_kwargs={"use_reentrant": False},
    # Use adamw_hf which better supports parameter groups for decoupled learning rates
    optim="adamw_hf"
)

# 6. Load model for examination to identify all module names
print("Loading model to identify layer structure...")
temp_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_cache=False,
    torch_dtype=torch.float16,
    device_map={"": 0}  # Load on first GPU only temporarily
)

# Identify embedding and other linear layers for targeting
embedding_layers = []
linear_layers = []
for name, module in temp_model.named_modules():
    if 'embed' in name.lower() or 'lm_head' in name.lower():
        embedding_layers.append(name)
    elif isinstance(module, torch.nn.Linear) and 'embed' not in name.lower() and 'lm_head' not in name.lower():
        linear_layers.append(name)

print(f"Found {len(embedding_layers)} embedding layers: {embedding_layers}")
print(f"Found {len(linear_layers)} linear layers")

# Clean up memory
del temp_model
torch.cuda.empty_cache()
gc.collect()

# 7. Comprehensive LoRA configuration for continued pretraining
# Combine both standard target_modules and add embedding layers
# List adjusted based on Qwen2.5 specific architecture
lora_config = LoraConfig(
    r=8,                            # Reduced from 16
    lora_alpha=16,                  # Reduced from 32
    target_modules=[                # Only target key modules
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj", "wte", "lm_head"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# 8. Initialize model with optimized settings
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_cache=False, # Load in fp16 for further efficiency
    device_map="auto"
)

# 9. Apply LoRA to the model
print("Applying LoRA adapters to model...")
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# 10. Create custom optimizer with decoupled learning rates
def get_optimizer_grouped_parameters(model, embedding_lr=1e-5, non_embedding_lr=1e-4):
    """Create parameter groups with different learning rates for embeddings vs other layers."""
    no_decay = ["bias", "LayerNorm.weight"]
    embedding_names = ["wte", "lm_head"]
    
    optimizer_grouped_parameters = [
        # Embedding params with lower learning rate and no weight decay
        {
            "params": [p for n, p in model.named_parameters() 
                      if any(nd in n for nd in embedding_names) and p.requires_grad],
            "lr": embedding_lr,
            "weight_decay": 0.0,
        },
        # Non-embedding params with regular learning rate and weight decay
        {
            "params": [p for n, p in model.named_parameters() 
                      if not any(nd in n for nd in embedding_names) 
                      and not any(nd in n for nd in no_decay) and p.requires_grad],
            "lr": non_embedding_lr,
            "weight_decay": training_args.weight_decay,
        },
        # Non-embedding params with regular learning rate and no weight decay
        {
            "params": [p for n, p in model.named_parameters() 
                      if not any(nd in n for nd in embedding_names) 
                      and any(nd in n for nd in no_decay) and p.requires_grad],
            "lr": non_embedding_lr,
            "weight_decay": 0.0,
        },
    ]
    return optimizer_grouped_parameters

# 11. Create a custom trainer with decoupled learning rates
class CustomPEFTTrainer(Trainer):
    def create_optimizer(self):
        """Create optimizer with separate learning rates for embedding vs. non-embedding parameters"""
        if self.optimizer is None:
            # Create parameter groups with decoupled learning rates
            embedding_lr = self.args.learning_rate / 10  # Lower embedding LR (10x smaller)
            non_embedding_lr = self.args.learning_rate
            
            print(f"Using decoupled learning rates: embedding={embedding_lr}, other={non_embedding_lr}")
            
            optimizer_grouped_parameters = get_optimizer_grouped_parameters(
                self.model, 
                embedding_lr=embedding_lr,
                non_embedding_lr=non_embedding_lr
            )
            
            # Create optimizer with parameter groups
            self.optimizer = torch.optim.AdamW(
                optimizer_grouped_parameters,
                lr=self.args.learning_rate,
                betas=(0.9, 0.999),
                eps=1e-8,
            )
        
        return self.optimizer

# 12. Initialize custom trainer with all optimizations
progress_callback = EnhancedProgressCallback()
trainer = CustomPEFTTrainer(
    model=model,
    args=training_args,
    train_dataset=chunked_dataset,
    data_collator=data_collator,
)
trainer.add_callback(progress_callback)

tokenizer_config.json:   0%|          | 0.00/7.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Maximum context length: 131072


...

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Found 2 embedding layers: ['model.embed_tokens', 'lm_head']
Found 196 linear layers
Applying LoRA adapters to model...


/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


trainable params: 10,460,160 || all params: 1,554,174,464 || trainable%: 0.6730


In [3]:

# Print training configuration
print(f"\n{'*'*70}")
print(f"ENHANCED LORA CONTINUED PRETRAINING CONFIGURATION:")
print(f"Model: {model_name}")
print(f"LoRA rank: {lora_config.r}")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Using decoupled learning rates: embedding={training_args.learning_rate/10}, other={training_args.learning_rate}")
print(f"Batch size: {training_args.per_device_train_batch_size} × "
      f"{training_args.gradient_accumulation_steps} steps × "
      f"{torch.cuda.device_count()} GPUs = "
      f"{training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps * torch.cuda.device_count()}")
print(f"Dataset size: {len(chunked_dataset)} chunks of {chunk_size} tokens each")
print(f"{'*'*70}\n")

# Start the training process
print("\n🚀 Starting research-optimized LoRA continued pretraining...\n")
try:
    trainer.train()
    print("\n✅ Training completed successfully!")
    # Save the final model
    print("Saving final model...")
    model.save_pretrained("./qwen_math_nbody_final_lora")
    tokenizer.save_pretrained("./qwen_math_nbody_final_lora")
    print("Model saved at ./qwen_math_nbody_final_lora")
except Exception as e:
    print(f"\n❌ Training interrupted: {e}")
    # Save checkpoint even if interrupted
    print("Saving emergency checkpoint...")
    model.save_pretrained("./qwen_math_nbody_checkpoint_lora")
    tokenizer.save_pretrained("./qwen_math_nbody_checkpoint_lora")
    print("Emergency checkpoint saved at ./qwen_math_nbody_checkpoint_lora")



**********************************************************************
ENHANCED LORA CONTINUED PRETRAINING CONFIGURATION:
Model: Qwen/Qwen2.5-Math-1.5B
LoRA rank: 8
Epochs: 3
Using decoupled learning rates: embedding=1e-05, other=0.0001
Batch size: 1 × 16 steps × 1 GPUs = 16
Dataset size: 752 chunks of 2048 tokens each
**********************************************************************


🚀 Starting research-optimized LoRA continued pretraining...

Using decoupled learning rates: embedding=1e-05, other=0.0001

TRAINING STARTED

Beginning Epoch 1/3


Epoch 1/3:   0%|          | 0/47 [00:00<?, ?it/s]

Step,Training Loss
1,14.962700
10,16.685900
20,16.294700
30,15.994400
40,16.111600
50,15.436500
60,16.073000
70,15.193600
80,15.488100
90,15.592300


Step: 20/141 (14.2%) | Loss: 16.2947 | LR: 2.00e-06
Step: 40/141 (28.4%) | Loss: 16.1116 | LR: 4.00e-06

Completed Epoch 1/3
Epoch time: 0:57:14
Total training time: 0:57:14
Estimated time remaining: 1:54:28


Beginning Epoch 2.0/3


Epoch 2.0/3:   0%|          | 0/47 [00:00<?, ?it/s]

...

Step: 120/141 (85.1%) | Loss: 15.1827 | LR: 5.12e-06
Step: 140/141 (99.3%) | Loss: 14.9034 | LR: 2.44e-07


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")



Completed Epoch 3.0/3
Epoch time: 0:57:19
Total training time: 2:51:49
Estimated time remaining: 0:00:00


✅ Training completed successfully!
Saving final model...


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


Model saved at ./qwen_math_nbody_final_lora
